In [1]:
import ckks
import numpy as np

In [2]:
ckks.XI

(0.7071067811865476+0.7071067811865475j)

In [3]:
M = 8 # Well use the roots of the M-th cyclotomic polynomial for encoding and decoding
N = M//2 # We are guaranteed the M-th cyclotomic is of degree M/2 if M is a power of 2

# XI is our complex root of unity, and want to work with the M-th
XI = np.exp(2 * np.pi * 1j / M)


In [4]:
encoder = ckks.CKKSEncoder(M)

In [5]:
v = np.array([1,2,3,4])

In [6]:
p = encoder.encode(v)

In [7]:
print(p)

(2.5+4.440892098500626e-16j) -
(4.996003610813204e-16-0.7071067811865479j)·x -
(3.4694469519536176e-16-0.5000000000000003j)·x² -
(8.326672684688674e-16-0.7071067811865472j)·x³


In [8]:
v_decoded = encoder.decode(p)

In [9]:
print(v_decoded) # we can the result of decoding is very close to original but with some error

[1.-7.66567226e-17j 2.-3.37225768e-16j 3.+8.49335536e-18j
 4.+7.47733715e-17j]


In [10]:
# distance from orig
np.linalg.norm(v_decoded - v)

7.208960525149399e-16

# Homomorphic operations
## Addition

In [11]:
v1 = np.array([1,2,3,4])
v2 = np.array([1,-2,3,-4])

In [12]:
p1 = encoder.encode(v1)
p2 = encoder.encode(v2)

In [13]:
v1_p_v2 = p1 + p2 # close to [0,0,0,0]
v1_m_v2 = p1 - p2 # close to 2*v1


In [14]:
print(np.round(encoder.decode(v1_p_v2)))
print(np.round(encoder.decode(v1_m_v2)))

[2.-0.j 0.-0.j 6.+0.j 0.+0.j]
[-0.-0.j  4.-0.j -0.-0.j  8.-0.j]


## Multiplication
Need a polynomial modulus. I believe in CKKKS there is a far more complex renormalization operations that
ensures polynomial products are computed using the modulus accurately but for now well just perform mod
in a straight forward manner

In [15]:
from numpy.polynomial import Polynomial
poly_mod = Polynomial([1,0,0,0,1]) # X^4 + 1, so polynomial products will never exceed degree 3

In [16]:
v1_prod_v2 = p1 * p2 % poly_mod

In [17]:
print(np.round(encoder.decode(v1_prod_v2)))

[  1.-0.j  -4.+0.j   9.+0.j -16.-0.j]


# Part 2 stuff

In [18]:
z = np.array([0,1])

# should double to [0,1,1,0]
z_pi = encoder.pi_inverse(z)
print(z_pi)

# should half to [0,1]
print(encoder.pi(z_pi))

[0 1 1 0]
[0 1]


In [19]:
print(encoder.sigma_R_basis)

[[ 1.00000000e+00+0.j          1.00000000e+00+0.j
   1.00000000e+00+0.j          1.00000000e+00+0.j        ]
 [ 7.07106781e-01+0.70710678j -7.07106781e-01+0.70710678j
  -7.07106781e-01-0.70710678j  7.07106781e-01-0.70710678j]
 [ 2.22044605e-16+1.j         -4.44089210e-16-1.j
   1.11022302e-15+1.j         -1.38777878e-15-1.j        ]
 [-7.07106781e-01+0.70710678j  7.07106781e-01+0.70710678j
   7.07106781e-01-0.70710678j -7.07106781e-01-0.70710678j]]


In [20]:
# check that linear combination of sigma basis is encoded as an integer polynomial
coords1 = [1,0,0,0]
coords2 = [1,1,1,1]
coords3 = [2,2,2,0]

In [21]:
b1 = np.matmul(encoder.sigma_R_basis.T,coords1)
b2 = np.matmul(encoder.sigma_R_basis.T,coords2)
b3 = np.matmul(encoder.sigma_R_basis.T,coords3)

In [22]:
print(b1)
print(b2)
print(b3)

[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
[1.+2.41421356j 1.+0.41421356j 1.-0.41421356j 1.-2.41421356j]
[3.41421356+3.41421356j 0.58578644-0.58578644j 0.58578644+0.58578644j
 3.41421356-3.41421356j]


In [25]:
print(encoder.encode(b1),"\n\n")
print(encoder.encode(b2),"\n\n")
print(encoder.encode(b3),"\n\n")

(1+0j) + (-0+0j)·x + (-0+0j)·x² + 0j·x³ 


(1+2.220446049250313e-16j) + (1+0j)·x +
(0.9999999999999998+2.7755575615628716e-17j)·x² +
(1+2.220446049250313e-16j)·x³ 


(2-2.220446049250313e-16j) + (2+4.440892098500626e-16j)·x +
(2-3.72797834410252e-17j)·x² -
(3.661623269986289e-16+1.04865410606504e-16j)·x³ 




In [26]:
# A non linear combination should not encode to a integer polynomial
coords4 = [1.5,1.5,1.5,1.5]
b4 = np.matmul(encoder.sigma_R_basis.T,coords4)
print(encoder.encode(b4))

(1.5000000000000004+0j) + (1.5+4.440892098500626e-16j)·x +
(1.5-1.387778780781426e-17j)·x² +
(1.4999999999999998-1.1102230246251565e-16j)·x³
